## 원본

In [1]:
import requests
from bs4 import BeautifulSoup

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')
        code = stock.select_one('.tltle').attrs['href'][-6:]

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            codes.append(code)
        else:
            pass
    except:
        continue
        
    if len(codes) == 5:
        break
        
print(codes)

['001570', '009270', '006220', '010580', '096300']


## 작업 중.. 
### (codes에 담긴 종목 중 변동성 돌파 기반으로 매수해야 할 종목이 있는지 추가적으로 데이터 크롤링 후 시각화 예정)

In [3]:
import requests
from bs4 import BeautifulSoup

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

In [4]:
stocks[0].select('.tah.p11')

[<span class="tah p11 red02">
 				30
 				</span>,
 <span class="tah p11 red01">
 				+0.95%
 				</span>]

In [5]:
stocks[0].select('.tah.p11')[0].text

'\n\t\t\t\t30\n\t\t\t\t'

In [6]:
stocks[0].select('.tah.p11')[1].text

'\n\t\t\t\t+0.95%\n\t\t\t\t'

In [7]:
import re

In [8]:
a = stocks[0].select('.tah.p11')[0].text
b = stocks[0].select('.tah.p11')[1].text
re_a = re.sub('[a-zA-Z]', '', a)
re_b = re.sub('[a-zA-Z]', '', b)
print(re_a, re_b)


				30
				 
				+0.95%
				


In [9]:
Notag_a = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', a)
Notag_b = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', b)
print(Notag_a, Notag_b)

30 +0.95%


In [36]:
ccc = stocks[1].select('td.number')[3:-2]
print(ccc, stocks[1].select('td.number')[3].text, stocks[1].select('td.number')[-3].text, sep = '\n')
ccc[0].text

[<td class="number">63,913,877</td>, <td class="number">33,485</td>, <td class="number">505</td>, <td class="number">506</td>, <td class="number">1,326</td>]
63,913,877
1,326


'63,913,877'

In [35]:
pp = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*|, ', '', str(ccc[0]))
pp

'63,913,877'

In [14]:
print(type(ccc[0]))

<class 'bs4.element.Tag'>


In [15]:
result = []

for cc in ccc:
    c = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*|, ', '', str(cc))
    print(cc)
    result.append(c)
result # 거래량, 거래대금, 매수호가, 매도호가, 시가총액

<td class="number">63,913,877</td>
<td class="number">33,485</td>
<td class="number">505</td>
<td class="number">506</td>
<td class="number">1,326</td>


['63,913,877', '33,485', '505', '506', '1,326']

In [124]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []
keys = ['종목명', '현재가', '전일비', '등락률', '거래량', '시가총액']
result = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            
            code = stock.select_one('.tltle').attrs['href'][-6:]
            codes.append(code)
            
            updown = stock.select('.tah.p11')[0].text
            percentage = stock.select('.tah.p11')[1].text
            
            re_updown = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', updown)
            re_percentage = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', percentage)
            
            market = stock.select('td.number')[3].text
            volumn = stock.select('td.number')[-3].text
            
            values = [stock_n, price, re_updown, re_percentage, market, volumn]
            
            info_dict = dict(zip(keys, values))
            result.append(info_dict)
        
        else:
            pass
    except:
        continue
        
    if len(codes) == 5:
        break
        
print(codes)

for r in result:
    print()
    print(r)

['001570', '009270', '006220', '010580', '096300']

{'종목명': '금양', '현재가': '17450', '전일비': '600', '등락률': '+3.56%', '거래량': '25,566,113', '시가총액': '10,048'}

{'종목명': '신원', '현재가': '1680', '전일비': '55', '등락률': '+3.38%', '거래량': '16,567,236', '시가총액': '1,606'}

{'종목명': '제주은행', '현재가': '7020', '전일비': '730', '등락률': '+11.61%', '거래량': '11,798,306', '시가총액': '2,255'}

{'종목명': '에스엠벡셀', '현재가': '2700', '전일비': '405', '등락률': '+17.65%', '거래량': '11,708,144', '시가총액': '3,004'}

{'종목명': '베트남개발1', '현재가': '71', '전일비': '5', '등락률': '+7.58%', '거래량': '11,487,373', '시가총액': '98'}


In [123]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []
keys = ['종목명', '현재가', '전일비', '등락률', '거래량', '시가총액']
result = []
urls = []

for stock in stocks:
    try:
        company = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')

        if (('인버스' not in company) and ('레버리지' not in company)) and (int(price) <= 26000):
            
            code = stock.select_one('.tltle').attrs['href'][-6:] # 종목코드 가져오기
            codes.append(code)
            
            updown = stock.select('.tah.p11')[0].text # 전일비 가져오기
            percentage = stock.select('.tah.p11')[1].text # 등락률 가져오기
            
            re_updown = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', updown) # 전일비 정규표현식으로 클렌징
            re_percentage = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', percentage) # 등락률 정규표현식으로 클렌징
            
            market = stock.select('td.number')[3].text # 거래량 가져오기
            volumn = stock.select('td.number')[-3].text # 시가총액 가져오기
            
            values = [company, price, re_updown, re_percentage, market, volumn]
            
            info_dict = dict(zip(keys, values))
            result.append(info_dict)
            
            sise_URL = 'https://finance.naver.com/item/sise_day.naver?code=' + code # 각 종목의 시세 탭으로 진입
            urls.append(sise_URL)
#             sise_res = requests.get(sise_URL)
        
        else:
            pass
    except:
        continue
        
    if len(codes) == 5:
        break
        
print(codes, [type(i) for i in codes])

for r in result:
    print()
    print(r)

print(*urls, sep='\n')

# for url in urls:
#     res = requests.get(url)
#     soup = BeautifulSoup(res.text, 'html.parser')

['001570', '009270', '006220', '010580', '096300'] [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]

{'종목명': '금양', '현재가': '17450', '전일비': '600', '등락률': '+3.56%', '거래량': '25,566,113', '시가총액': '10,048'}

{'종목명': '신원', '현재가': '1680', '전일비': '55', '등락률': '+3.38%', '거래량': '16,567,236', '시가총액': '1,606'}

{'종목명': '제주은행', '현재가': '7020', '전일비': '730', '등락률': '+11.61%', '거래량': '11,798,306', '시가총액': '2,255'}

{'종목명': '에스엠벡셀', '현재가': '2700', '전일비': '405', '등락률': '+17.65%', '거래량': '11,708,144', '시가총액': '3,004'}

{'종목명': '베트남개발1', '현재가': '71', '전일비': '5', '등락률': '+7.58%', '거래량': '11,487,373', '시가총액': '98'}
https://finance.naver.com/item/sise_day.naver?code=001570
https://finance.naver.com/item/sise_day.naver?code=009270
https://finance.naver.com/item/sise_day.naver?code=006220
https://finance.naver.com/item/sise_day.naver?code=010580
https://finance.naver.com/item/sise_day.naver?code=096300


In [131]:
headers= {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:104.0) Gecko/20100101 Firefox/104.0'}
sise_URL = 'https://finance.naver.com/item/sise_day.naver?code=001570'
sise_res = requests.get(sise_URL, headers=headers)
# print(res.status_code)

soup = BeautifulSoup(sise_res.text, 'html.parser')
prices = soup.select('.type2 tr')[2]

'''
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
'''

latest_date = prices.select_one('.tah.p10.gray03').text
start_price = prices.select('.tah.p11')[2].text.replace(',', '') # 전날 시가이기 때문에 필요 없음. 당일 시가로 대체해야
highest_price = prices.select('.tah.p11')[3].text.replace(',', '')
lowest_price = prices.select('.tah.p11')[4].text.replace(',', '')
target_price = int(start_price) + (int(highest_price) - int(lowest_price)) * 0.5
print(prices, latest_date, start_price, highest_price, lowest_price, int(target_price), sep = '\n')
print(16700 + (21250-16250) * 0.5) # 돌파계수 K가 default 0.5일 때. 매수 타켓가격: 19200
print(16700 + (21250-16250) * 0.85) # 돌파계수 K가 시장 상황에 따라 0.85일 때. 매수 타켓가격: 20950

<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2022.09.08</span></td>
<td class="num"><span class="tah p11">17,450</span></td>
<td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				600
				</span>
</td>
<td class="num"><span class="tah p11">16,700</span></td>
<td class="num"><span class="tah p11">21,250</span></td>
<td class="num"><span class="tah p11">16,250</span></td>
<td class="num"><span class="tah p11">25,566,113</span></td>
</tr>
2022.09.08
16700
21250
16250
19200
19200.0
20950.0


In [110]:
print(1-abs(16700-17450)/(21250-16250))
print(16700-17450); print(21250-16250); print(750/5000)

0.85
-750
5000
0.15


In [134]:
from datetime import datetime, timedelta

today = datetime.now().date()
yesterday = today - timedelta(days=1)
# print(str(yesterday).replace('-', '.'))
yesterday = str(yesterday).replace('-', '.')
print(yesterday)
prices = soup.select('table.type2')
prices

2022.09.08


[<table cellspacing="0" class="type2">
 <tr>
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
 </tr>
 <tr>
 <td colspan="7" height="8"></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2022.09.08</span></td>
 <td class="num"><span class="tah p11">17,450</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
 				600
 				</span>
 </td>
 <td class="num"><span class="tah p11">16,700</span></td>
 <td class="num"><span class="tah p11">21,250</span></td>
 <td class="num"><span class="tah p11">16,250</span></td>
 <td class="num"><span class="tah p11">25,566,113</span></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2022.09.07</span></td>
 <td class="num"><span class="t